# Navigation

---

In this notebook, you will learn how to use the Unity ML-Agents environment for the first project of the [Deep Reinforcement Learning Nanodegree](https://www.udacity.com/course/deep-reinforcement-learning-nanodegree--nd893).

### 1. Start the Environment

We begin by importing some necessary packages.  If the code cell below returns an error, please revisit the project instructions to double-check that you have installed [Unity ML-Agents](https://github.com/Unity-Technologies/ml-agents/blob/master/docs/Installation.md) and [NumPy](http://www.numpy.org/).

In [1]:
from unityagents import UnityEnvironment
import numpy as np

Next, we will start the environment!  **_Before running the code cell below_**, change the `file_name` parameter to match the location of the Unity environment that you downloaded.

- **Mac**: `"path/to/Banana.app"`
- **Windows** (x86): `"path/to/Banana_Windows_x86/Banana.exe"`
- **Windows** (x86_64): `"path/to/Banana_Windows_x86_64/Banana.exe"`
- **Linux** (x86): `"path/to/Banana_Linux/Banana.x86"`
- **Linux** (x86_64): `"path/to/Banana_Linux/Banana.x86_64"`
- **Linux** (x86, headless): `"path/to/Banana_Linux_NoVis/Banana.x86"`
- **Linux** (x86_64, headless): `"path/to/Banana_Linux_NoVis/Banana.x86_64"`

For instance, if you are using a Mac, then you downloaded `Banana.app`.  If this file is in the same folder as the notebook, then the line below should appear as follows:
```
env = UnityEnvironment(file_name="Banana.app")
```

In [2]:
env = UnityEnvironment(file_name="env/Banana_Linux_NoVis/Banana.x86")

INFO:unityagents:
'Academy' started successfully!
Unity Academy name: Academy
        Number of Brains: 1
        Number of External Brains : 1
        Lesson number : 0
        Reset Parameters :
		
Unity brain name: BananaBrain
        Number of Visual Observations (per agent): 0
        Vector Observation space type: continuous
        Vector Observation space size (per agent): 37
        Number of stacked Vector Observation: 1
        Vector Action space type: discrete
        Vector Action space size (per agent): 4
        Vector Action descriptions: , , , 


Environments contain **_brains_** which are responsible for deciding the actions of their associated agents. Here we check for the first brain available, and set it as the default brain we will be controlling from Python.

In [3]:
# get the default brain
brain_name = env.brain_names[0]
brain = env.brains[brain_name]

### 2. Examine the State and Action Spaces

The simulation contains a single agent that navigates a large environment.  At each time step, it has four actions at its disposal:
- `0` - walk forward 
- `1` - walk backward
- `2` - turn left
- `3` - turn right

The state space has `37` dimensions and contains the agent's velocity, along with ray-based perception of objects around agent's forward direction.  A reward of `+1` is provided for collecting a yellow banana, and a reward of `-1` is provided for collecting a blue banana. 

Run the code cell below to print some information about the environment.

In [4]:
# reset the environment
env_info = env.reset(train_mode=False)[brain_name]

# number of agents in the environment
print('Number of agents:', len(env_info.agents))

# number of actions
action_size = brain.vector_action_space_size
print('Number of actions:', action_size)

# examine the state space 
state = env_info.vector_observations[0]
print('States look like:', state)
state_size = len(state)
print('States have length:', state_size)

Number of agents: 1
Number of actions: 4
States look like: [1.         0.         0.         0.         0.84408134 0.
 0.         1.         0.         0.0748472  0.         1.
 0.         0.         0.25755    1.         0.         0.
 0.         0.74177343 0.         1.         0.         0.
 0.25854847 0.         0.         1.         0.         0.09355672
 0.         1.         0.         0.         0.31969345 0.
 0.        ]
States have length: 37


### 3. Take Random Actions in the Environment

In the next code cell, you will learn how to use the Python API to control the agent and receive feedback from the environment.

Once this cell is executed, you will watch the agent's performance, if it selects an action (uniformly) at random with each time step.  A window should pop up that allows you to observe the agent, as it moves through the environment.  

Of course, as part of the project, you'll have to change the code so that the agent is able to use its experience to gradually choose better actions when interacting with the environment!

env.close()

### 4. It's Your Turn!

Now it's your turn to train your own agent to solve the environment!  When training the environment, set `train_mode=True`, so that the line for resetting the environment looks like the following:
```python
env_info = env.reset(train_mode=True)[brain_name]
```

In [5]:
from agent import DqnAgent

In [6]:
import torch

In [7]:
import torch
from torch import nn
from torch.nn import functional
import random
import numpy as np
# from torch.utils.tensorboard import SummaryWriter


device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")


# Experiment hyperparameters
LR = 0.001
EPS = 0.1
ALPHA = 0.2
GAMMA = 0.99

# tensorboard
# writer = SummaryWriter()

class DqnAgent():
    '''Agent that will interact with our environnement'''

    def __init__(self, state_size, action_size, seed=None):
        if seed:
            self.seed = random.seed(seed)

        self.state_size = state_size
        self.action_size = action_size
        
        # Network
        self.qnetwork = QNetwork(state_size=state_size, action_size=action_size).to(device)
        self.optimizer = torch.optim.SGD(self.qnetwork.parameters(), lr=LR)

    def learn(self, transition_info, done):
        '''
            Update the policy of the agent from one step infos.
            Input:
                transition_info(tuple): Tuple of the format (s, a, r, s')
        '''
        state, action, reward, next_state = transition_info
        
        # Get old estimate of the q_values according to doing old action while being in the old state. 
        Qva = self.qnetwork(state)
        
        # Get new estimated q values in states s' while still following policy.
        max_espected_value = torch.max(self.qnetwork(next_state).detach())
        
        # if done, then there is no next Q value estimate to make.
        max_espected_value = (1 - done) * max_espected_value
        
        update_loss = Qva + ALPHA * (reward + GAMMA * max_espected_value - Qva)
        
        # reseting optimizer at each update step
        self.optimizer.zero_grad()
        Qva.backward(update_loss.data.unsqueeze(1))

        # update
        self.optimizer.step()

        
    def act(self, state):
        '''
            Select an action based on inference from the state following an e-greedy policy.
            Input:
                state(np.Array): The representation of the state from the environnement.
            Return:
                action(int): Action
        '''
        actions = self.qnetwork.forward(state)

        if random.random() > EPS:
            # Select highest Q values action
            action = np.argmax(np.argmax(actions.detach().numpy()))
        else:
            # Select a random actions
            action = random.choice(np.arange(self.action_size))

        return action


class QNetwork(torch.nn.Module):
    """Actor (Policy) Model."""

    def __init__(self, state_size, action_size, **params):
        super(QNetwork, self).__init__()

        # self state input, 64 output
        self.lin1 = nn.Linear(state_size, 64)
        self.lin2 = nn.Linear(64, 64)
        self.lin3 = nn.Linear(64, action_size)

        self.criterion = nn.CrossEntropyLoss()

    def forward(self, state):
        # convert state to tensor
        state = torch.from_numpy(state).float().to(device)  # unsqueeze ?

        x = functional.relu(self.lin1(state))
        x = functional.relu(self.lin2(x))
        x = functional.relu(self.lin3(x))

        return x


In [8]:
network = DqnAgent(state_size=len(state), action_size=action_size)

In [ ]:

scores = []
loses = []

# Instanciate a new agent.
agent = DqnAgent(state_size=len(state), action_size=action_size)

for episode in range(200):
    done = False
    score = 0
    
    env_info = env.reset(train_mode=True)[brain_name]
    action_size = brain.vector_action_space_size
    state = env_info.vector_observations[0]

    while True:
        action = agent.act(state)                              # ask agent action selection
        env_info = env.step(action)[brain_name]                # send the action to the environment

        next_state = env_info.vector_observations[0]           # get the next state
        reward = env_info.rewards[0]                           # get the reward
        done = env_info.local_done[0]                          # see if episode has finished
        score += reward                                        # update the score
        transition_info = (state, action, reward, next_state)

        agent.learn(transition_info, done)                     # learn from transition
        score += reward

        if done:
            scores.append(score)
            break

        state = next_state                                     # roll over the state to next time step

    if episode % 10 == 0:
        print('Episode: {} - '.format(episode), end="")
        print('Averaged Score {}'.format(np.mean(scores).round(2)))

# env.close()

# Tensorboard
#     writer.add_scalar('Averaged Score', np.mean(scores), episode)


Episode: 0 - Averaged Score 0.0
Episode: 10 - Averaged Score 0.73
